# Load books

In [1]:
import glob
import os
books_directory = '/home/ubuntu/Downloads/twiker-experiments/process_analysis/articles/books'
books = []

# List all .txt files in the directory using glob
book_files = glob.glob(os.path.join(books_directory, '*.txt'))
# Iterate over each file in the directory
for book_file in book_files:
    with open(book_file, 'r') as fs:
        books.append(fs.read())
    print(f'{os.path.basename(book_file)}', len(books[-1]))
print()
print(books[0][0:1000])

article_000491.txt 3614
article_000606.txt 1526
article_000522.txt 2322
article_000728.txt 2549
article_000457.txt 1370
article_000233.txt 1527
article_000326.txt 3544
article_000620.txt 2411
article_000434.txt 2202
article_000348.txt 5176
article_000464.txt 1133
article_000319.txt 5472
article_000538.txt 6085
article_000207.txt 1777
article_000462.txt 4581
article_000278.txt 3406
article_000388.txt 2267
article_000421.txt 3181
article_000624.txt 5695
article_000594.txt 6386
article_000500.txt 1657
article_000661.txt 2753
article_000343.txt 5190
article_000256.txt 4076
article_000267.txt 2753
article_000374.txt 3046
article_000600.txt 3903
article_000660.txt 2118
article_000428.txt 3708
article_000558.txt 4175
article_000742.txt 4977
article_000770.txt 1240
article_000356.txt 4372
article_000655.txt 982
article_000322.txt 3792
article_000425.txt 4607
article_000260.txt 1939
article_000328.txt 2956
article_000383.txt 5577
article_000372.txt 1505
article_000366.txt 4120
article_000495.tx

In [2]:
import re
def clean_text(text):
    """Clean the text by removing special characters, tags, etc."""
    # Replace newlines and carriage returns with a space
    #text = text.replace('\n', ' ').replace('\r', ' ')

    # Remove all non-alphanumeric characters except spaces
    #text = re.sub(r'[^A-Za-z0-9\s]+', '', text)
    #text = re.sub('<[^>]+>%/', '', text)
    text = re.sub('<[^>]+>', ' ', text) # Remove any character that is not a letter, number, punctuation, or whitespace 
    #text = re.sub(r'[^A-Za-z0-9\s.,!?;:()-]', '', text)
    text = re.sub(r'[^\w\s.,]',' ',text)
    text = re.sub(r'\.\s*\|\s*\.',' ',text)
    #text = text.replace('·', ' ')
    #text = text.replace('|',' ')
    #text = text.replace('|||||',' ')
    text = re.sub(r'[\u00A0\u2009]', ' ',text)

 
    # Remove leading tabs and extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

cleaned_books =[clean_text(book) for book in books]

In [23]:
# Some long paragraphs will be truncated during training. 
# To avoid lossing data, we split them into shorter ones.

import nltk
from transformers import GPT2Tokenizer

# Download the necessary NLTK data
nltk.download('punkt')

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def split_text_at_sentences(text, max_tokens=400):
    # Tokenize the text into sentences
    sentences = nltk.tokenize.sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_tokens = []
    
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        
        # Check if adding this sentence would exceed the max token limit
        if len(current_tokens) + len(sentence_tokens) > max_tokens:
            # If so, finalize the current chunk and start a new one
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_tokens = []
        
        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_tokens.extend(sentence_tokens)
    
    # Add the last chunk if there's any content left
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
chucked_books = []
for i, book in enumerate(cleaned_books):
    chunked_book = ''
    chunks = split_text_at_sentences(book.replace("\n", " "))
    for chunk in chunks:
        chunked_book += chunk + "\n"
    chucked_books.append(chunked_book)

In [25]:
print(len(chucked_books))

598


# Merge to train and eval

In [36]:
num_train_books = 390
train = ''
eval =''
for i in range(num_train_books):
    train += chucked_books[i]
for j in range(len(chucked_books)-num_train_books):    
    eval += chucked_books[num_train_books+j]

with open(f'data_files/train.txt', 'w') as fs:
    fs.write(train)

with open(f'data_files/eval.txt', 'w') as fs:
    fs.write(eval)

with open(f'data_files/full.txt', 'w') as fs:
    fs.write(train + eval)

In [35]:
def count_lines_in_file(filename):
    with open(filename, 'r') as file:
        line_count = sum(1 for line in file)
    return line_count
 
# Example usage:
filename = 'your_text_file.txt'
train_number_of_lines = count_lines_in_file('data_files/train.txt')
full_number_of_lines = count_lines_in_file('data_files/full.txt')
eval_number_of_lines = count_lines_in_file('data_files/eval.txt')
print(f'The train file has {train_number_of_lines} lines.')
print(f'The full file has {full_number_of_lines} lines.')
print(f'The eval file has {eval_number_of_lines} lines.')

The train file has 1017 lines.
The full file has 1669 lines.
The eval file has 652 lines.
